In [1]:
import numpy as np 
import pandas as pd 
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/brain-tumors-256x256/Data/pituitary_tumor/P_343.jpg
/kaggle/input/brain-tumors-256x256/Data/pituitary_tumor/P_739.jpg
/kaggle/input/brain-tumors-256x256/Data/pituitary_tumor/P_182.jpg
/kaggle/input/brain-tumors-256x256/Data/pituitary_tumor/P_661.jpg
/kaggle/input/brain-tumors-256x256/Data/pituitary_tumor/P_199.jpg
/kaggle/input/brain-tumors-256x256/Data/pituitary_tumor/P_20.jpg
/kaggle/input/brain-tumors-256x256/Data/pituitary_tumor/P_288.jpg
/kaggle/input/brain-tumors-256x256/Data/pituitary_tumor/P_83.jpg
/kaggle/input/brain-tumors-256x256/Data/pituitary_tumor/P_842.jpg
/kaggle/input/brain-tumors-256x256/Data/pituitary_tumor/P_612.jpg
/kaggle/input/brain-tumors-256x256/Data/pituitary_tumor/P_284.jpg
/kaggle/input/brain-tumors-256x256/Data/pituitary_tumor/P_305.jpg
/kaggle/input/brain-tumors-256x256/Data/pituitary_tumor/P_829.jpg
/kaggle/input/brain-tumors-256x256/Data/pituitary_tumor/P_393.jpg
/kaggle/input/brain-tumors-256x256/Data/pituitary_tumor/P_98.jpg
/kaggle/input

In [2]:
import numpy as np 
import pandas as pd
import pathlib as plt
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
import tensorflow as tf
from tensorflow.keras import utils,layers, Sequential
import cv2

# Configure GPU

In [3]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)

# Get and Classify Data

In [4]:
url = "/kaggle/input/brain-tumors-256x256/Data/"

In [5]:
data = pathlib.Path(url)

NameError: name 'pathlib' is not defined

In [ ]:
disease = {
    "glioma_tumor": list(data.glob('glioma_tumor/*.jpg')),
    "meningioma_tumor": list(data.glob('meningioma_tumor/*.jpg')),
    "normal": list(data.glob('normal/*.jpg')),
    "pituitary_tumor" : list(data.glob('pituitary_tumor/*.jpg')),       

}

In [ ]:
labels = {
    "glioma_tumor" : 0,
    "meningioma_tumor":1,
    "normal":2,
    "pituitary_tumor":3
}

In [ ]:
print(str(disease["glioma_tumor"][:5]))

# Display Data

In [ ]:
def display_sample_images(sample_title, num_samples=5):
    plt.figure(figsize=(12, 4))
    img_lst = disease[sample_title][:num_samples]
    for i, image in enumerate(img_lst):
        img = mpimg.imread(str(image))
        plt.subplot(1, num_samples, i+1)
        plt.imshow(img, cmap='gray')
        plt.title(sample_title)
        plt.axis('off')
    plt.show()
    
display_sample_images("glioma_tumor")
display_sample_images("meningioma_tumor")
display_sample_images("normal")
display_sample_images("pituitary_tumor")
    



# Format Data

In [ ]:
X = []
Y = []
for name, diseases in disease.items():
    for disease in diseases:
        img = cv2.imread(str(disease))
        img = img.astype('float32') / 255.0
        X.append(img)
        Y.append(labels[name])     
        
        

In [ ]:
X = np.array(X)
Y = np.array(Y)

In [ ]:
X.shape


In [ ]:
from sklearn.model_selection import train_test_split 
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,random_state=42,stratify=Y)

# Build Model

In [ ]:
model = Sequential([
    #CNN layer
    layers.Conv2D(16,(3,3),padding="same",activation="relu",input_shape=(256,256,3)),
    layers.MaxPooling2D(),
    #ANN layer
    layers.Flatten(),
    layers.Dense(256,activation="relu"),
    layers.Dropout(0.2),
    layers.Dense(4,activation='softmax')
])


In [ ]:
model.compile(
    optimizer='adam',
    loss  = 'sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Train Model

In [ ]:
model.fit(X_train,Y_train,epochs=5)

# Test Model

In [ ]:
model.evaluate(X_test,Y_test)

In [ ]:
model.summary()

# Diagnosising MRI Inputs From Users

In [ ]:
mri_scan = input("Please Input Downloaded File Location")
#Example: /kaggle/input/images-of-tumor/unknown_tumor.jpg
img = cv2.imread(mri_scan)
img = img.astype('float32') / 255.0
img = cv2.resize(img, (256, 256))
img = np.array([img])

In [ ]:
tumor_prediction = model.predict(img)

In [ ]:
tumor_labels = ["Glioma Tumor", "Meningioma Tumor", "No Tumors", "Pituitary Tumor"]
highest_probability = np.argmax(tumor_prediction)
diagnosis = tumor_labels[highest_probability]

In [ ]:
print(f"Based on the provided MRI, the model has an 80% level of confidence that the diagnosis is {diagnosis}.")